<a href="https://colab.research.google.com/github/shyoonCS/DataAnalysis/blob/main/Report202502.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

과제 1-2. API 호출코드 작성

In [55]:
import requests
import pandas as pd

url = "http://openapi.seoul.go.kr:8088/4446516e7973687936367073734f62/json/energyUseDataSummaryInfo/1/5"
api_key = '4446516e7973687936367073734f62'
year_from = 2015
year_to = 2024

params = {
    'KEY': api_key,
    'TYPE': 'json',
    'SERVICE': 'energyUseDataSummaryInfo',
    'START_INDEX': '1',
    'END_INDEX': '9999',
    'YEAR': '2024',
    'MON': '12'
}

## 년도 생성
def get_year_months():
  year_months = []
  for year in range(year_from, year_to+1):
    for month in range(1,12+1):
      year_months.append((f'{year}', f'{month:02d}'))
  return year_months

data = []
# URL 생성
for year_month in get_year_months() :
  url2 = url + '/' + year_month[0] + '/' + year_month[1]
  #print(url2)

  ## 데이터 수집
  response = requests.get(url2, params=params)
  data = data + response.json()['energyUseDataSummaryInfo']['row']

## 호출 성공/실패 출력
#print(data)

df = pd.DataFrame(data)
df.to_csv('data.csv', index=False, encoding='utf-8')
df.to_excel('data.xlsx', index=False)